### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.


In [1]:
import pandas
from HorsesService import HorsesService
import numpy

In [2]:
horsesService = HorsesService()
horse_data_frame = horsesService.fetch_processed_hourse_data_frame()

In [3]:
def get_horse_data(full_horse_data_frame: pandas.DataFrame) -> pandas.DataFrame:
    table = full_horse_data_frame[[
        'rectal_temperature',
        'pulse',
        'respiratory_rate',
        'temperature_of_extremities',
        'mucous_membranes',
        'outcome',
        'packed_cell_volume',
        'nasogastric_reflux'
    ]]
    table_copy = table.copy()
    keys = ['mucous_membranes', 'outcome', 'nasogastric_reflux', 'nasogastric_reflux']
    table_copy[keys] = table[keys].replace('nan', numpy.nan)
    return table_copy

horse_data = get_horse_data(
    full_horse_data_frame=horse_data_frame
)

Убираем все данные с 4 и более пропусками

In [4]:
horse_data.dropna(
    thresh=len(horse_data.columns) - 3
)


,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,mucous_membranes,outcome,packed_cell_volume,nasogastric_reflux
0,38.5,66.0,28.0,3.0,NaN,2,45.0,NaN
1,39.2,88.0,20.0,NaN,4,3,50.0,NaN
2,38.3,40.0,24.0,1.0,3,1,33.0,NaN
3,39.1,164.0,84.0,4.0,6,2,48.0,2
4,37.3,104.0,35.0,NaN,6,2,74.0,NaN
...,...,...,...,...,...,...,...,...
295,NaN,120.0,70.0,4.0,4,3,55.0,NaN
296,37.2,72.0,24.0,3.0,4,3,44.0,1
297,37.5,72.0,30.0,4.0,4,2,60.0,1
298,36.5,100.0,24.0,3.0,3,1,50.0,1


Заполняем пропуски медианой

In [5]:
rectal_temperature_median = horse_data['rectal_temperature'].median()
horse_data['rectal_temperature'] = horse_data['rectal_temperature'].fillna(rectal_temperature_median)

def fill_median(
        horse_data: pandas.DataFrame,
        columns: [str],
        is_round: bool = False
) -> pandas.DataFrame:
    for column_key in columns:
        column_median = None
        try:
            quantile_1 = horse_data[column_key].quantile(.15)
            quantile_3 = horse_data[column_key].quantile(.85)
            between_table = horse_data[horse_data[column_key].between(quantile_1, quantile_3)]
            outliers_table = pandas.concat([horse_data, between_table]).drop_duplicates(keep=False)
            column_median = outliers_table[column_key].median()
        except:
            column_median = horse_data[column_key].median()

        if is_round:
            column_median = column_median.round()

        horse_data[column_key] = horse_data[column_key].fillna(column_median)

fill_median(
    horse_data=horse_data,
    columns=[
        'rectal_temperature',
        'pulse',
        'respiratory_rate',
        'packed_cell_volume'
    ]
)

fill_median(
    horse_data=horse_data,
    columns=[
        'temperature_of_extremities',
        'outcome',
        'mucous_membranes',
        'nasogastric_reflux'
    ],
    is_round=True
)
horse_data

,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,mucous_membranes,outcome,packed_cell_volume,nasogastric_reflux
0,38.5,66.0,28.0,3.0,3.0,2,45.0,1.0
1,39.2,88.0,20.0,4.0,4,3,50.0,1.0
2,38.3,40.0,24.0,1.0,3,1,33.0,1.0
3,39.1,164.0,84.0,4.0,6,2,48.0,2
4,37.3,104.0,35.0,4.0,6,2,74.0,1.0
...,...,...,...,...,...,...,...,...
295,38.2,120.0,70.0,4.0,4,3,55.0,1.0
296,37.2,72.0,24.0,3.0,4,3,44.0,1
297,37.5,72.0,30.0,4.0,4,2,60.0,1
298,36.5,100.0,24.0,3.0,3,1,50.0,1
